# MVD 10. cvičení
V dnešním cvičení rozšíříme BERT model o jednu FCNN vrstvu, která se dotrénuje na úloze klasifikace sentimentu. Bude se vycházet z předtrénovaného BERT modelu a přidaná vrstva se bude trénovat s Pytorch knihovnou.

In [ ]:
import random
import torch
import numpy as np
import pandas as pd

In [ ]:
# Parametry trénování
epochs = 1
lr = 1e-6
batch_size = 16

## 1. část - Příprava dat pro detekci sentimentu

Stáhněte si data filmových recenzí na [Kaggle datasets](https://www.kaggle.com/columbine/imdb-dataset-sentiment-analysis-in-csv-format) a rozbalte je do adresáře data. Spusťte a projděte následující kód.

In [ ]:
# Vybrána menší množina dat -> lze spustit i na cpu
train_df = pd.read_csv('data/Train.csv').head(1000)
train_df['type'] = 'train'
test_df = pd.read_csv('data/Test.csv').head(100)
test_df['type'] = 'test'

data_df = pd.concat([train_df, test_df])

print(data_df.shape)
print(data_df.head())
print(data_df.text.iloc[0])
# Kontrola vyvážení datasetu
print(data_df.label.value_counts())

In [ ]:
# Nastavení random seedu pro možnost reprodukce experimentu
seed = 42
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)
encoder_train = tokenizer.batch_encode_plus(data_df[data_df['type']=='train'].text.values,
                                           add_special_tokens = True,
                                           padding = True,
                                           truncation=True,
                                           return_tensors = 'pt')

encoder_test = tokenizer.batch_encode_plus(data_df[data_df['type']=='test'].text.values,
                                           add_special_tokens = True,
                                           padding = True,
                                           truncation=True,
                                           return_tensors = 'pt')

input_ids_train = encoder_train['input_ids']
attention_masks_train = encoder_train["attention_mask"]
labels_train = torch.tensor(data_df[data_df['type'] == 'train'].label.values)

input_ids_test = encoder_test['input_ids']
attention_masks_test = encoder_test["attention_mask"]
labels_test = torch.tensor(data_df[data_df['type'] == 'test'].label.values)

### Attention masks
Attention masky používáme z důvodu paddingu (doplnění nul do určité délky). Attention maska obsahuje 0 nebo 1 a říká modelu, kde se nachází původní vstup a kde padding, na který se nemá zaměřovat. 


In [ ]:
from torch.utils.data import RandomSampler, SequentialSampler, DataLoader

data_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
data_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

dataloader_train = DataLoader(data_train, shuffle = True, batch_size = batch_size)
dataloader_test = DataLoader(data_test, batch_size = batch_size)

## 2. část - Příprava modelu a trénování

In [ ]:
from transformers import BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 2)

optimizer = AdamW(model.parameters(), lr = lr)
scheduler = get_linear_schedule_with_warmup(
                optimizer,
                num_warmup_steps = 0,
                num_training_steps = len(dataloader_train)*epochs 
            )

### BertForSequenceClassification
Tato třída již obsahuje jednu přidanou lineární vrstvu se 2 výstupními třídami (viz parametr num_labels). Tuto vrstvu uvidíte na konci seznamu vrstev.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device: {}".format(device))
model.to(device)

In [ ]:
from tqdm.notebook import tqdm

for epoch in tqdm(range(epochs)):
    model.train()
    train_loss = 0
    
    with tqdm(dataloader_train, desc = f'Epoch: {epoch}') as pbar:
        for batch in pbar:
            model.zero_grad()

            inputs = {
                "input_ids" : batch[0].to(device),
                "attention_mask" : batch[1].to(device),
                "labels" : batch[2].to(device)
            }
            outputs = model(**inputs)

            loss = outputs[0]
            # outputs[1] -> logits
            train_loss += loss.item()
            loss.backward()

            optimizer.step()
            scheduler.step()

            pbar.set_postfix({'training_loss':f'{(loss.item() / len(batch)):.3f}'})
    
    loss_train_avg = train_loss / len(dataloader_train)
    tqdm.write(f'Training Loss: {loss_train_avg}')

## 3. Část - Vyhodnocení
Vytvořte funkci `eval`, která bude přijímat jako vstupní parametr test (nebo valid) DataLoader. V této funkci projdete data v evaluačním módu modelu, spočítáte loss na daných datech a využijete výstup `output[1] # logits` pro výpočet úspěšnosti (accuracy) a f1-skóre (pomocí sklearn funkce).

In [ ]:
from sklearn.metrics import f1_score 
